<a href="https://colab.research.google.com/github/ayamlearning/zambia-2021-elections-prediction/blob/main/Step_One_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install emoji
!pip install contractions
!pip install inflect

In [ ]:
import re, string, unicodedata
import nltk
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
import pandas as pd
import emoji
nltk.download('punkt')

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Masters Program/data/sample.csv',
               index_col=0)
df.head()

In [ ]:
def remove_URL(sample):
    """Remove URLs from a sample string"""
    return re.sub(r"http\S+", "", sample)

In [ ]:
def convert_emojis_to_word(text):
    """Removing emojis from the text"""
    return emoji.demojize(text)

In [ ]:
def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)

In [ ]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

In [ ]:
def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

In [ ]:
def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

In [ ]:
def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

In [ ]:
def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

In [ ]:
def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

In [ ]:
def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

In [ ]:
def remove_mentions_and_hashtags(sample):
  sample = re.sub("@[A-Za-z0-9_]+","", sample)
  sample = re.sub("#[A-Za-z0-9_]+","", sample)
  return sample

In [ ]:
def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    #words = remove_punctuation(words)
    words = replace_numbers(words)
    #words = remove_stopwords(words)
    return " ".join(words)

In [ ]:
def preprocess(sample):
    sample = remove_URL(sample)
    sample = replace_contractions(sample)
    sample = convert_emojis_to_word(sample)
    sample = remove_mentions_and_hashtags(sample)
    # Tokenize
    words = nltk.word_tokenize(sample)
    # Normalize
    return normalize(words)

In [ ]:
preprocess("""
Our very own sweet Manana 😍😍
Zambia’s First Lady @MrsHichilema 
#ZambiaDecides2021 #ZambiaDecided https://t.co/e9rh6DwdZU""") 
  



In [ ]:
df['text']=df['text'].apply(preprocess)
df.head()

In [ ]:
df.to_csv('/content/drive/MyDrive/Masters Program/data/clean_sample.csv')